## Amenazas Transformation

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

## Read File

In [4]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO '] 

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"Doesn't find the header line in: {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

In [9]:
file_paths = glob.glob(os.path.join(PATH_FILE+'/Amenazas/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+'/Amenazas/','*.xls'))
dataframes = []

for file_path in file_paths:
    try:
        df = read_excel_with_variable_header(file_path)
        dataframes.append(df)
    except ValueError as e:
        print(e)

## Clean File

In [13]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

standardized_dataframes = [standardize_columns(df) for df in dataframes]

In [14]:
amenazas_df = pd.concat(standardized_dataframes, ignore_index=True)

In [20]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','GRANADA DE MANO',
                             'NO REPORTADO','SIN EMPLEO DE ARMAS','ESCOPOLAMINA']
amenazas_df["AGRUPA_EDAD_PERSONA"] = amenazas_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
amenazas_df["GENERO"] = amenazas_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
amenazas_df["ARMAS_MEDIOS"] = amenazas_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [28]:
amenazas_df['FECHA_TEMP'] = pd.to_datetime(amenazas_df['FECHA'])
amenazas_df['FECHA'] = amenazas_df['FECHA_TEMP'].dt.date
amenazas_df.drop(columns=['FECHA_TEMP'], inplace=True)

In [33]:
amenazas_output = amenazas_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS"]).sum().reset_index()

## Export File

In [34]:
amenazas_output.to_csv(OUTPUT_PATH_FILE+"/amenazas.csv",index=False)